In [1]:
import findspark
findspark.init()
import pyspark
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()


In [2]:
lines = spark\
        .readStream\
        .format('socket')\
        .option('host', "localhost")\
        .option('port', 9999)\
        .load()
words = lines.select(
        # explode turns each item in an array into a separate row
    explode(
        split(lines.value, ' ')
    ).alias('word')
)

wordCounts = words.groupBy('word').count()

In [3]:
query = wordCounts\
    .writeStream\
    .queryName("wordCounts")\
    .outputMode('complete')\
    .format('memory')\
    .start()

In [9]:
display(spark.sql(f"SELECT * FROM {query.name}").show())

+-------+-----+
|   word|count|
+-------+-----+
|    you|    1|
|   cock|    1|
| sucker|    1|
|fucking|    1|
+-------+-----+



None